Script para subir os dados do censo escolar de 2024

Apenas a tabela `escola`


In [1]:
import os
import sys

In [2]:
if "google.colab" in sys.modules:
    %pip install -q "basedosdados[upload]"
    from google.colab import drive  # type: ignore

    drive.mount("/content/drive")
    ROOT = "/content/drive/MyDrive/Base dos Dados <> Fundação Lemann/Demandas/SAEB - Análise - Redes Pares/PSM"
else:
    ROOT = os.getcwd()

In [3]:
import pandas as pd
import numpy as np
import csv
import basedosdados as bd
from io import StringIO
import requests
import zipfile

In [4]:
INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

In [12]:
os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [5]:
ZIP_FILE = os.path.join(INPUT, "microdados_censo_escolar_2024.zip")

In [16]:
with zipfile.ZipFile(ZIP_FILE, "r") as zip_ref:
    zip_ref.extractall(INPUT)

In [6]:
csv_path = os.path.join(
    INPUT, "microdados_censo_escolar_2024", "dados", "microdados_ed_basica_2024.csv"
)

In [7]:
def detect_delimiter(file_path):
    with open(file_path, "r", encoding="iso-8859-1") as file:
        sample = file.read(1024)  # Read a sample of the file
        sniffer = csv.Sniffer()
        delimiter = sniffer.sniff(sample).delimiter
        return delimiter


detected_delimiter = detect_delimiter(csv_path)
print(f"The detected delimiter is: {detected_delimiter}")

The detected delimiter is: ;


In [8]:
censo = pd.read_csv(
    csv_path,
    delimiter=";",
    dtype="string",
    encoding="iso-8859-1",
)

In [9]:
censo_raw_columns = censo.columns.to_list()

In [10]:
censo.shape

(215545, 426)

In [11]:
censo.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_REGIAO_GEOG_INTERM,CO_REGIAO_GEOG_INTERM,...,QT_TUR_BAS_D,QT_TUR_BAS_N,QT_TUR_BAS_EAD,QT_TUR_INF_INT,QT_TUR_INF_CRE_INT,QT_TUR_INF_PRE_INT,QT_TUR_FUND_INT,QT_TUR_FUND_AI_INT,QT_TUR_FUND_AF_INT,QT_TUR_MED_INT
0,2024,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Ji-Paraná,1102,...,4,0,0,0,0,0,0,0,0,0
1,2024,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Ji-Paraná,1102,...,0,4,0,0,0,0,0,0,0,0
2,2024,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Ji-Paraná,1102,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2024,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Ji-Paraná,1102,...,10,0,0,0,0,0,0,0,0,0
4,2024,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Ji-Paraná,1102,...,35,5,0,0,0,0,0,0,0,0


In [12]:
def apply_architecture_to_dataframe(
    df: pd.DataFrame,
    url_architecture: str,
    apply_rename_columns: bool = True,
    apply_column_order_and_selection: bool = True,
    apply_include_missing_columns: bool = True,
):
    """
    Transforms a DataFrame based on the specified architecture.

    Args:
        df (pandas DataFrame): The input DataFrame.
        url_architecture (str): The URL of the architecture.
        apply_rename_columns (bool, optional): Flag to apply column renaming. Defaults to True.
        apply_column_order_and_selection (bool, optional): Flag to apply column order and selection. Defaults to True.
        apply_include_missing_columns (bool, optional): Flag to include missing columns. Defaults to True.

    Returns:
        pandas DataFrame: The transformed DataFrame.

    Raises:
        Exception: If an error occurs during the transformation process.
    """
    architecture = read_architecture_table(url_architecture=url_architecture)

    if apply_rename_columns:
        df = rename_columns(df, architecture)

    if apply_include_missing_columns:
        df = include_missing_columns(df, architecture)

    if apply_column_order_and_selection:
        df = column_order_and_selection(df, architecture)

    return df


def read_architecture_table(url_architecture: str) -> pd.DataFrame:
    """URL contendo a tabela de arquitetura no formato da base dos dados
    Args:
        url_architecture (str): url de tabela de arquitetura no padrão da base dos dados
    Returns:
        df: um df com a tabela de arquitetura
    """
    # Converte a URL de edição para um link de exportação em formato csv
    url = url_architecture.replace("edit#gid=", "export?format=csv&gid=")

    # Coloca a arquitetura em um dataframe
    df_architecture = pd.read_csv(
        StringIO(requests.get(url, timeout=10).content.decode("utf-8"))
    )

    df_architecture.query("name != '(excluido)'", inplace=True)

    return df_architecture.replace(np.nan, "", regex=True)


def get_order(architecture: pd.DataFrame) -> list:
    """
    Retrieves the column order from an architecture table.
    Args:
        architecture (pd.DataFrame): The architecture table containing column information.
    Returns:
        list: The list of column names representing the order.
    """

    # Return the list of column names from the 'name' column of the architecture table
    return list(architecture["name"])


def rename_columns(df: pd.DataFrame, architecture: pd.DataFrame) -> pd.DataFrame:
    """
    Renames the columns of a DataFrame based on an architecture table.
    Args:
        df (pd.DataFrame): The DataFrame to rename columns.
        architecture (pd.DataFrame): The architecture table containing column mappings.
    Returns:
        pd.DataFrame: The DataFrame with renamed columns.
    """

    # Create a DataFrame 'aux' with unique mappings of column names from the architecture table
    aux = architecture[["name", "original_name"]].drop_duplicates(
        subset=["original_name"], keep=False
    )

    # Create a dictionary 'dict_columns' with column name mappings
    dict_columns = dict(zip(aux.original_name, aux.name))

    # Rename columns of the DataFrame 'df' based on the dictionary 'dict_columns'
    return df.rename(columns=dict_columns)


def include_missing_columns(df, architecture):
    """
    Includes missing columns in the DataFrame based on the specified architecture.

    Args:
        df (pandas DataFrame): The input DataFrame.
        architecture (str): The specified architecture.

    Returns:
        pandas DataFrame: The modified DataFrame with missing columns included.
    """
    df_missing_columns = missing_columns(df.columns, get_order(architecture))
    if df_missing_columns:
        df[df_missing_columns] = ""
        print(f"The following columns were included into the df: {df_missing_columns}")
    else:
        print("No columns were included into the df")
    return df


def missing_columns(current_columns, specified_columns):
    """
    Determines the missing columns between the current columns and the specified columns.

    Args:
        current_columns (list): The list of current columns.
        specified_columns (list): The list of specified columns.

    Returns:
        list: The list of missing columns.
    """
    missing_columns = []
    for col in specified_columns:
        if col not in current_columns:
            missing_columns.append(col)

    return missing_columns


def column_order_and_selection(df, architecture):
    """
    Performs column order and selection on the DataFrame based on the specified architecture.

    Args:
        df (pandas DataFrame): The input DataFrame.
        architecture (str): The specified architecture.

    Returns:
        pandas DataFrame: The DataFrame with columns ordered and selected according to the architecture.
    """
    architecture_columns = get_order(architecture)
    list_missing_columns = missing_columns(
        current_columns=architecture_columns, specified_columns=df
    )
    if list_missing_columns:
        print(
            f"The following columns were discarded from the df: {list_missing_columns}"
        )
    else:
        print("No columns were discarded from the df")
    return df[architecture_columns]

In [13]:
url_architecture = "https://docs.google.com/spreadsheets/d/1WmKRJjOmcG9uFL0LaBx4EwZUA_o2VpZK2MO3hFfTnmM/edit#gid=0"

In [14]:
censo = apply_architecture_to_dataframe(
    df=censo,
    url_architecture=url_architecture,
    apply_rename_columns=True,
    apply_column_order_and_selection=True,
    apply_include_missing_columns=True,
)

The following columns were included into the df: ['conveniada_poder_publico', 'tipo_convenio_poder_publico', 'tipo_atendimento_escolarizacao', 'tipo_atendimento_atividade_complementar', 'tipo_atendimento_aee', 'local_funcionamento_sala_empresa', 'local_funcionamento_templo_igreja', 'local_funcionamento_casa_professor', 'agua_filtrada', 'energia_gerador', 'energia_outros', 'lixo_joga_outra_area', 'lixo_outros', 'lixo_reciclagem', 'banheiro_fora_predio', 'banheiro_dentro_predio', 'bercario', 'dependencia_pne', 'lavanderia', 'quantidade_sala_existente', 'quantidade_equipamento_parabolica', 'quantidade_equipamento_copiadora', 'quantidade_equipamento_impressora', 'quantidade_equipamento_impressora_multifuncional', 'equipamento_videocassete', 'quantidade_equipamento_videocassete', 'equipamento_retroprojetor', 'quantidade_equipamento_retroprojetor', 'equipamento_fax', 'quantidade_equipamento_fax', 'equipamento_foto', 'quantidade_equipamento_foto', 'quantidade_computador_aluno', 'quantidade_co

In [16]:
censo.head()

,ano,sigla_uf,id_municipio,id_distrito,id_escola,rede,tipo_categoria_escola_privada,tipo_localizacao,tipo_localizacao_diferenciada,tipo_situacao_funcionamento,...,quantidade_turma_profissional_tecnica,quantidade_turma_eja,quantidade_turma_eja_fundamental,quantidade_turma_eja_medio,quantidade_turma_especial,quantidade_turma_especial_comum,quantidade_turma_especial_exclusiva,quantidade_turma_diurno,quantidade_turma_noturno,quantidade_turma_ead
0,2024,RO,1100015,110001505,11022558,2,<NA>,2,2,1,...,0,0,0,0,0,0,0,4,0,0
1,2024,RO,1100015,110001505,11024275,2,<NA>,1,0,1,...,0,4,2,2,2,2,0,0,4,0
2,2024,RO,1100015,110001505,11024291,3,<NA>,2,<NA>,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2024,RO,1100015,110001505,11024666,3,<NA>,2,0,1,...,0,0,0,0,7,7,0,10,0,0
4,2024,RO,1100015,110001505,11024682,2,<NA>,1,0,1,...,0,0,0,0,16,16,0,35,5,0


In [18]:
bq_censo_cols = bd.read_sql(
    "select * from `basedosdados.br_inep_censo_escolar.escola` limit 0",
    billing_project_id="basedosdados-dev",
).columns.to_list()

Downloading: |          |


In [20]:
censo_2023_cols = bd.read_sql(
    "select * from `basedosdados-dev.br_inep_censo_escolar_staging.escola_2023` limit 0",
    billing_project_id="basedosdados-dev",
).columns.to_list()

Downloading: |          |


In [21]:
len(censo_2023_cols)

455

In [22]:
len(bq_censo_cols)

455

In [23]:
len(censo.columns)

456

In [24]:
len(censo_raw_columns)

426

In [26]:
# Colunas que devemos remover porque não estão na tabela no BQ
cols_to_drop = [i for i in censo.columns if i not in bq_censo_cols]

In [27]:
cols_to_drop

['id_distrito']

In [28]:
censo = censo.drop(columns=cols_to_drop)

In [29]:
len(bq_censo_cols) == len(censo.columns)

True

In [30]:
SAVE_DIR = os.path.join(OUTPUT, "escola", "ano=2024")
os.makedirs(SAVE_DIR, exist_ok=True)

In [32]:
for sigla_uf, df_uf in censo.groupby("sigla_uf"):
    path = os.path.join(SAVE_DIR, f"sigla_uf={sigla_uf}")
    os.makedirs(path, exist_ok=True)
    df_uf.drop(columns=["ano", "sigla_uf"]).to_csv(
        os.path.join(path, "data.csv"), index=False
    )

In [33]:
tb = bd.Table(dataset_id="br_inep_censo_escolar", table_id="escola_2024")

In [35]:
tb.create(os.path.join(OUTPUT, "escola"))

Uploading files: 100%|██████████| 27/27 [00:53<00:00,  1.97s/it]
2025-04-09 14:39:41.419 | SUCCESS  | basedosdados.upload.table:create:690 - Table escola_2024 was created in staging!
